In [1]:
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import tikzplotlib
import seaborn as sns

import json
from datetime import datetime

In [2]:
projects_df = pd.read_csv('/root/data/projects.csv',
                         parse_dates=['project_created_at', 'project_last_pushed_at', 'project_updated_at'])
projects_df['project_revision'] = projects_df.apply(lambda x: x['project_revision'][:10], axis=1)
packages_df = pd.read_csv('/root/data/packages_0_499.csv').dropna()
geiger_df = pd.read_csv('/root/data/geiger/geiger_findings_0_499.csv')
sampled_usages_app = pd.read_csv('/root/data/classification/sampled_usages_app.csv')
sampled_usages_std = pd.read_csv('/root/data/classification/sampled_usages_std.csv')

# Evaluation 1: with labeled data set

In [101]:
index = 39

snippet = sampled_usages_app[sampled_usages_app['label']=='cast-header'].iloc[index]

print("{}:{} in package {} of module {}\n".format(
    snippet['file_name'], int(snippet['line_number']), snippet['package_import_path'], snippet['module_path']))

print("{}@{}/{}/{}\n".format(snippet['module_path'], snippet['module_version'], 
                          snippet['package_import_path'][len(snippet['module_path'])+1:], snippet['file_name']))

print(snippet['context'])

bytestostr.go:21 in package github.com/mailru/easyjson/jlexer of module github.com/mailru/easyjson

github.com/mailru/easyjson@v0.7.0/jlexer/bytestostr.go

//
// Warning: the string returned by the function should be used with care, as the whole input data
// chunk may be either blocked from being freed by GC because of a single string or the buffer.Data
// may be garbage-collected even when the string exists.
func bytesToStr(data []byte) string {
	h := (*reflect.SliceHeader)(unsafe.Pointer(&data))
	shdr := reflect.StringHeader{Data: h.Data, Len: h.Len}
	return *(*string)(unsafe.Pointer(&shdr))
}


# Evaluation 2: with manually analyzed projects